<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/Deep-Learning-CNN-Projects/blob/master/CIFAR100-Stacked-Generalization-Image-Classification/Advanced_Stacking_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Configuration

In [3]:
class Config:
    IMG_SIZE = (64, 64)
    NUM_CLASSES = 100
    BATCH_SIZE = 64
    EPOCHS_BASE = 10
    EPOCHS_META = 20

config = Config()

# Data Preparation

In [12]:
# Loading CIFAR100
(x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

# Normalizing pixel values
x_train_full = x_train_full.astype('float32')
x_test = x_test.astype('float32')

# Convert labels to One-Hot
y_train_full = to_categorical(y_train_full, config.NUM_CLASSES)
y_test = to_categorical(y_test, config.NUM_CLASSES)

In [13]:
# We split training data into two parts:
# 1. Base Train: To train ResNet, EfficientNet, MobileNet
# 2. Meta Train (Hold-out): To generate predictions for the Meta-Learner
x_base, x_meta, y_base, y_meta = train_test_split(
    x_train_full, y_train_full, test_size=0.2, random_state=42)

print(f"Base Train Shape: {x_base.shape}")
print(f"Meta Train Shape: {x_meta.shape}")
print(f"Test Set: {x_test.shape}")

Base Train Shape: (40000, 32, 32, 3)
Meta Train Shape: (10000, 32, 32, 3)
Test Set: (10000, 32, 32, 3)


In [15]:
# Augmentation Layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Resizing images
def resize_img(x, y):
    return tf.image.resize(x, config.IMG_SIZE), y

def create_ds(x, y, shuffle=False, augment=False):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(resize_img, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds


# Create Pipelines
ds_base = create_ds(x_base, y_base, shuffle=True, augment=True)
ds_meta = create_ds(x_meta, y_meta, shuffle=False, augment=False)
ds_test = create_ds(x_test, y_test, shuffle=False, augment=False)

# Define Base Models

In [21]:
def build_base_model(model_name):
    inputs = layers.Input(shape=(64, 64, 3))

    # Explicit Preprocessing Layers
    if model_name == 'ResNet50':
        # ResNet expects inputs roughly around [-127, 127] centered
        # Rescaling(1.) means no division, but we need to ensure inputs are 0-255
        # We use standard keras preprocessing layer which is safer
        x = layers.Rescaling(1.0)(inputs)
        x = tf.keras.applications.resnet50.preprocess_input(x)
        base = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=x)

    elif model_name == 'EfficientNetB0':
        # EfficientNet expects [0, 255], it handles scaling internally
        x = inputs
        base = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_tensor=x)

    elif model_name == 'MobileNetV2':
        # MobileNetV2 expects [-1, 1]. So we divide by 127.5 and subtract 1
        x = layers.Rescaling(1./127.5, offset=-1)(inputs)
        base = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_tensor=x)

    # Smart Freezing
    base.trainable = True
    for layer in base.layers[:-20]:
        layer.trainable = False


    for layer in base.layers:
        if isinstance(layer, layers.BatchNormalization):
            layer.trainable = False

    print(f"Built {model_name}: Only last 20 layers trainable.")

    # Robust Classification Head
    x = layers.GlobalAveragePooling2D()(base.output)

    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(config.NUM_CLASSES, activation='softmax')(x)

    model = models.Model(inputs, outputs, name=model_name)

    model.compile(optimizer=keras.optimizers.Adam(1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [22]:
model_names = ['ResNet50', 'EfficientNetB0', 'MobileNetV2']
trained_models = []

print("\n" + "="*50)
print("Phase 1: Training Base Models")
print("="*50)

for name in model_names:
    print(f"\nTraining {name}...")
    model = build_base_model(name)

    # Checkpoint to save best weights
    ckpt = keras.callbacks.ModelCheckpoint(
        f"{name}_cifar.keras", save_best_only=True, monitor='val_loss')

    model.fit(
        ds_base, epochs=config.EPOCHS_BASE, validation_data=ds_meta,
        callbacks=[ckpt], verbose=1)

    # Load best weights
    model.load_weights(f"{name}_cifar.keras")
    trained_models.append(model)

print("\nBase models trained successfully.")


Phase 1: Training Base Models

Training ResNet50...
🏗️ Built ResNet50: Only last 20 layers trainable.
Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 97s 133ms/step - accuracy: 0.1673 - loss: 3.9368 - val_accuracy: 0.4753 - val_loss: 1.9658
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 79s 124ms/step - accuracy: 0.4130 - loss: 2.3054 - val_accuracy: 0.5258 - val_loss: 1.7579
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 79s 123ms/step - accuracy: 0.4838 - loss: 1.9858 - val_accuracy: 0.5587 - val_loss: 1.6833
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 78s 123ms/step - accuracy: 0.5334 - loss: 1.7755 - val_accuracy: 0.5797 - val_loss: 1.5305
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 123ms/step - accuracy: 0.5687 - loss: 1.6091 - val_accuracy: 0.5884 - val_loss: 1.5172
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 79s 124ms/step - accuracy: 0.6020 - loss: 1.4688 - val_accuracy: 0.5962 - val_loss: 1.5054
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 78s 122ms/step - accuracy: 0.6286 - loss: 1.3588 - val_accuracy: 0.5939

/tmp/ipython-input-2663354291.py:21: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_tensor=x)


🏗️ Built MobileNetV2: Only last 20 layers trainable.
Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 79s 106ms/step - accuracy: 0.0826 - loss: 4.5985 - val_accuracy: 0.3213 - val_loss: 2.7157
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 61s 95ms/step - accuracy: 0.2697 - loss: 3.0316 - val_accuracy: 0.4294 - val_loss: 2.1863
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 94ms/step - accuracy: 0.3538 - loss: 2.5593 - val_accuracy: 0.4706 - val_loss: 1.9827
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 61s 94ms/step - accuracy: 0.4013 - loss: 2.3305 - val_accuracy: 0.4979 - val_loss: 1.8429
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 61s 94ms/step - accuracy: 0.4371 - loss: 2.1687 - val_accuracy: 0.5043 - val_loss: 1.7949
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 61s 94ms/step - accuracy: 0.4571 - loss: 2.0597 - val_accuracy: 0.5301 - val_loss: 1.7089
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 94ms/step - accuracy: 0.4764 - loss: 1.9773 - val_accuracy: 0.5338 - val_loss: 1.6996
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━

# Create Meta-Dataset

In [23]:
print("Phase 2: Generating Meta-Features")

def get_meta_features(models, dataset):
    """
    Feeds data through all base models and concatenates their predictions.
    Output Shape: (Samples, 3 * 100)
    """
    preds = []
    for model in models:
        print(f"Generating predictions from {model.name}...")
        p = model.predict(dataset, verbose=1)
        preds.append(p)

    # Stack predictions horizontally: [Pred_ResNet, Pred_EffNet, Pred_Mobile]
    # Shape: (N, 100) + (N, 100) + (N, 100) -> (N, 300)
    meta_features = np.concatenate(preds, axis=1)
    return meta_features

# Generate features for Meta Training (using Hold-out set)
print(">>> Processing Meta-Train Set...")
X_meta_train = get_meta_features(trained_models, ds_meta)
Y_meta_train = y_meta # Labels remain the same

# Generate features for Final Testing
print(">>> Processing Test Set...")
X_meta_test = get_meta_features(trained_models, ds_test)
Y_meta_test = y_test

print(f"\nMeta-Features Shape: {X_meta_train.shape}")

Phase 2: Generating Meta-Features
>>> Processing Meta-Train Set...
Generating predictions from ResNet50...
157/157 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step
Generating predictions from EfficientNetB0...
157/157 ━━━━━━━━━━━━━━━━━━━━ 14s 53ms/step
Generating predictions from MobileNetV2...
157/157 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step
>>> Processing Test Set...
Generating predictions from ResNet50...
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step
Generating predictions from EfficientNetB0...
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
Generating predictions from MobileNetV2...
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step

Meta-Features Shape: (10000, 300)


# Train Meta-Learner (The Judge)

In [24]:
print("Phase 3: Training Meta-Learner")

def build_meta_learner(input_dim):
    inputs = layers.Input(shape=(input_dim,))

    # Simple Neural Network
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(config.NUM_CLASSES, activation='softmax')(x)

    model = models.Model(inputs, outputs, name="Meta_Learner")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Build model
meta_model = build_meta_learner(input_dim=X_meta_train.shape[1])

# Train the judge
meta_history = meta_model.fit(
    X_meta_train, Y_meta_train,
    validation_data=(X_meta_test, Y_meta_test),
    epochs=config.EPOCHS_META,
    batch_size=32,
    verbose=1
)


Phase 3: Training Meta-Learner
Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.1907 - loss: 3.9244 - val_accuracy: 0.5599 - val_loss: 3.2612
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5810 - loss: 2.0920 - val_accuracy: 0.6523 - val_loss: 1.5084
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6151 - loss: 1.7017 - val_accuracy: 0.6552 - val_loss: 1.4173
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6340 - loss: 1.5643 - val_accuracy: 0.6561 - val_loss: 1.3968
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6355 - loss: 1.5048 - val_accuracy: 0.6576 - val_loss: 1.3887
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6505 - loss: 1.4241 - val_accuracy: 0.6582 - val_loss: 1.3776
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6528 - loss: 1.3914 - val_accuracy: 0.6603 - val_loss: 1.3873
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6517 -

# Evaluation

In [25]:
print("Final Results Comparison")
# X_meta_test structure: [ResNet(100) | EffNet(100) | Mobile(100)]

p_resnet = X_meta_test[:, :100]
p_effnet = X_meta_test[:, 100:200]
p_mobile = X_meta_test[:, 200:]

acc_resnet = accuracy_score(np.argmax(y_test, axis=1), np.argmax(p_resnet, axis=1))
acc_effnet = accuracy_score(np.argmax(y_test, axis=1), np.argmax(p_effnet, axis=1))
acc_mobile = accuracy_score(np.argmax(y_test, axis=1), np.argmax(p_mobile, axis=1))

# Simple Averaging (Voting) Accuracy
avg_preds = (p_resnet + p_effnet + p_mobile) / 3.0
acc_voting = accuracy_score(np.argmax(y_test, axis=1), np.argmax(avg_preds, axis=1))

# Stacking Accuracy
stack_preds = meta_model.predict(X_meta_test)
acc_stacking = accuracy_score(np.argmax(y_test, axis=1), np.argmax(stack_preds, axis=1))

print(f"ResNet50 Accuracy:        {acc_resnet:.4f}")
print(f"EfficientNetB0 Accuracy:  {acc_effnet:.4f}")
print(f"MobileNetV2 Accuracy:     {acc_mobile:.4f}")
print("-" * 30)
print(f"Simple Voting Accuracy:   {acc_voting:.4f}")
print(f"Stacking (Meta) Accuracy: {acc_stacking:.4f}")

if acc_stacking > max(acc_resnet, acc_effnet, acc_mobile):
    print("\nSUCCESS: Stacking outperformed base models!")
else:
    print("\nNote: Stacking requires well-tuned base models to shine.")

Final Results Comparison
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
ResNet50 Accuracy:        0.5914
EfficientNetB0 Accuracy:  0.5953
MobileNetV2 Accuracy:     0.5495
------------------------------
Simple Voting Accuracy:   0.6565
Stacking (Meta) Accuracy: 0.6521

SUCCESS: Stacking outperformed base models!
